In [1]:
from api import API
import pandas as pd

In [2]:
api = API()

## Load / Merge Data

In [3]:
bld = api.building('Hulings').id
timestamp = '2017-08-18 00:45:00'

# Load some data
rooms = api.building_rooms(bld)
points = api.building_points(bld)
vals = api.building_values_at_time(bld, timestamp)

In [4]:
# Remove room 6, the scary dummy room
rooms = rooms[rooms['id'] != 6]

In [5]:
# Merge df together
df = pd.merge(points, rooms, left_on='roomid', right_on='id',
              suffixes=('_point', '_room'))

df= pd.merge(df, vals, left_on='id_point', right_on='pointid',)

In [6]:
df.head(n=3)

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,112 RH VALVE,NaN,572,HU.R112.RHV,2,1,18,4,18,112,...,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.00,float,PSI,2017-08-18,00:45
1,112 TEMP,NaN,573,HU.R112.RM,2,13,18,4,18,112,...,13,floatDEG F2,573,HU.R112.RM,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
2,112 TEMP,NaN,574,HU.R112.RMT,2,13,18,4,18,112,...,13,floatDEG F2,574,HU.R112.RMT,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45


## Attempt to tag specific points

In [26]:
def pivot_table_around_tags(df, idx):
    """Pivot table so tags become columns"""

    # Get values where tag is not 'none'
    df = df.query('tag != "none"')

    # call pivot_on_tag with each
    tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']
    frames = [pivot_on_tag(df, t) for t in tags]
    final_df = pd.concat(frames, axis=1).reset_index()

    return final_df


def pivot_on_tag(df, tag):
    """
    Helper function to 'pivot_table_around_tags'.

    Takes: tagged df (with all rooms, points, and vals for given building/timestamp) and tag (eg, "valve")
    Returns: filtered/pivoted df: <index = name_room, columns = [pointid_{tag}, {tag}]>
    """
    return (df.query('tag == "{}"'.format(tag))
            .pivot_table(index=['name_room', 'pointid'], columns='tag', values='pointvalue')
            .reset_index().rename_axis(None, axis=1)
            .rename({'pointid': 'pointid_{}'.format(tag)}, axis='columns')
            .set_index('name_room'))

## Room Comapre

In [ ]:
df['tag'] = 'none'

# Tag 'valve' based on description
df_valve = df[df['description'].str.find('VALVE') > 0]
df.loc[df_valve.index, 'tag'] = 'valve'

# Tag 'temp' based on point name
df_temp = df[df['name_point'].str.find('.RM') > 0]
df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

df_temp = df[df['name_point'].str.find('.RMT') > 0]
df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'

In [25]:
def pivot_on_tag(df, tag):
    """
    Helper function to 'pivot_table_around_tags'.

    Takes: tagged df (with all rooms, points, and vals for given building/timestamp) and tag (eg, "valve")
    Returns: filtered/pivoted df. Index = name_room, Columns=[poin
    """
    return (df.query('tag == "{}"'.format(tag))
            .pivot_table(index=['name_room', 'pointid'], columns='tag', values='pointvalue')
            .reset_index().rename_axis(None, axis=1)
            .rename({'pointid': 'pointid_{}'.format(tag)}, axis='columns')
            .set_index('name_room'))

In [21]:
tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']

In [22]:
frames = [pivot_on_tag(df, t) for t in tags]

In [23]:
frames[0]

,pointid_valve,valve
name_room,,
102,608,12.0
112,572,3.0
121,589,9.0
202,622,15.0
203,640,9.0
206,631,9.0
207,662,9.0
212,668,3.0
213,682,9.0


In [17]:
final_df = pd.concat(frames, axis=1, join='inner').reset_index()

In [18]:
final_df

,name_room,pointid_valve,valve,pointid_temp1 (RM),temp1 (RM),pointid_temp2 (RMT),temp2 (RMT)
0,102,608,12.0,609,73.39,610,73.39
1,112,572,3.0,573,70.92,574,70.92
2,121,589,9.0,590,70.61,591,70.61
3,202,622,15.0,623,69.60,624,71.01
4,203,640,9.0,641,67.65,642,67.65
5,206,631,9.0,632,69.10,633,69.10
6,207,662,9.0,663,76.93,664,76.93
7,212,668,3.0,669,72.18,670,72.18
8,213,682,9.0,683,71.60,684,71.60
9,215,699,9.0,700,69.11,701,69.11


In [32]:
a = final_df.set_index('room_name')

,pointid_valve,valve,pointid_temp1 (RM),temp1 (RM),pointid_temp2 (RMT),temp2 (RMT)
room_name,,,,,,
"(102, 102, 102)",608,12.0,609,73.39,610,73.39
"(112, 112, 112)",572,3.0,573,70.92,574,70.92
"(121, 121, 121)",589,9.0,590,70.61,591,70.61
"(202, 202, 202)",622,15.0,623,69.60,624,71.01
"(203, 203, 203)",640,9.0,641,67.65,642,67.65
"(206, 206, 206)",631,9.0,632,69.10,633,69.10
"(207, 207, 207)",662,9.0,663,76.93,664,76.93
"(212, 212, 212)",668,3.0,669,72.18,670,72.18
"(213, 213, 213)",682,9.0,683,71.60,684,71.60


In [18]:
final_df.set_index('room_name')

,pointid_valve,valve,pointid_temp1 (RM),temp1 (RM),pointid_temp2 (RMT),temp2 (RMT)
room_name,,,,,,
"(102, 102, 102)",608,12.0,609,73.39,610,73.39
"(112, 112, 112)",572,3.0,573,70.92,574,70.92
"(121, 121, 121)",589,9.0,590,70.61,591,70.61
"(202, 202, 202)",622,15.0,623,69.60,624,71.01
"(203, 203, 203)",640,9.0,641,67.65,642,67.65
"(206, 206, 206)",631,9.0,632,69.10,633,69.10
"(207, 207, 207)",662,9.0,663,76.93,664,76.93
"(212, 212, 212)",668,3.0,669,72.18,670,72.18
"(213, 213, 213)",682,9.0,683,71.60,684,71.60


In [214]:
all_vals = api.building_values_in_range(4, '2017-08-16', '2017-08-19')

In [217]:
all_vals[all_vals.pointid.isin(final_df['pointid_valve'])]

,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
5856,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:00:00,3.0,float,PSI,2017-08-18,00:00
5857,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:15:00,3.0,float,PSI,2017-08-18,00:15
5858,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:30:00,3.0,float,PSI,2017-08-18,00:30
5859,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.0,float,PSI,2017-08-18,00:45
5860,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 01:00:00,3.0,float,PSI,2017-08-18,01:00
5861,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 01:15:00,3.0,float,PSI,2017-08-18,01:15
5862,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 01:30:00,3.0,float,PSI,2017-08-18,01:30
5863,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 01:45:00,3.0,float,PSI,2017-08-18,01:45
5864,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 02:00:00,3.0,float,PSI,2017-08-18,02:00
5865,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 02:15:00,3.0,float,PSI,2017-08-18,02:15


In [213]:
all_vals['pointid'] == [511, 707]

TypeError: invalid type comparison

In [139]:
# View DF where tag is not "none
view_df = df.query('tag == "valve"')

# Pivot 
view_df = ((view_df.pivot_table(
    index=['name_room', 'pointid'],
    columns='tag',
    values='pointvalue'))
           .reset_index().rename_axis(None, axis=1))

view_df

,name_room,pointid,valve
0,102,608,12.0
1,112,572,3.0
2,121,589,9.0
3,202,622,15.0
4,203,640,9.0
5,206,631,9.0
6,207,662,9.0
7,212,668,3.0
8,213,682,9.0
9,215,699,9.0


In [101]:
view_df.head()

tag                temp1 (RM)  temp2 (RMT)  valve
name_room pointid                                
102       608             NaN          NaN   12.0
          609           73.39          NaN    NaN
          610             NaN        73.39    NaN
112       572             NaN          NaN    3.0
          573           70.92          NaN    NaN

In [107]:
# View DF where tag is not "none
view_df = df.query('tag != "none"')

# Pivot 
view_df = (view_df.pivot(index='name_room', columns='tag',
                         values='pointvalue'))

view_df

tag,temp1 (RM),temp2 (RMT),valve
name_room,,,
102,73.39,73.39,12.0
112,70.92,70.92,3.0
121,70.61,70.61,9.0
202,69.60,71.01,15.0
203,67.65,67.65,9.0
206,69.10,69.10,9.0
207,76.93,76.93,9.0
212,72.18,72.18,3.0
213,71.60,71.60,9.0


In [84]:
view_df

tag,temp1 (RM),temp2 (RMT),valve
name_room,,,
102,73.39,73.39,12.0
112,70.92,70.92,3.0
121,70.61,70.61,9.0
202,69.60,71.01,15.0
203,67.65,67.65,9.0
206,69.10,69.10,9.0
207,76.93,76.93,9.0
212,72.18,72.18,3.0
213,71.60,71.60,9.0


In [91]:
view_df = df.query('tag == "valve"')

In [93]:
view_df.head()

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag
0,112 RH VALVE,NaN,572,HU.R112.RHV,2,1,18,4,18,112,...,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.0,float,PSI,2017-08-18,00:45,valve
7,121 RH VALVE,NaN,589,HU.R121.RHV,2,1,19,4,19,121,...,floatPSI2,589,HU.R121.RHV,2017-08-18 00:45:00,9.0,float,PSI,2017-08-18,00:45,valve
17,102 RH VALVE,NaN,608,HU.R102.RHV,2,1,20,4,20,102,...,floatPSI2,608,HU.R102.RHV,2017-08-18 00:45:00,12.0,float,PSI,2017-08-18,00:45,valve
26,202 RH VALVE,NaN,622,HU.R202.RHV,2,1,21,4,21,202,...,floatPSI2,622,HU.R202.RHV,2017-08-18 00:45:00,15.0,float,PSI,2017-08-18,00:45,valve
35,206 RH VALVE,NaN,631,HU.R206.RHV,2,1,22,4,22,206,...,floatPSI2,631,HU.R206.RHV,2017-08-18 00:45:00,9.0,float,PSI,2017-08-18,00:45,valve


In [43]:
# View DF where tag is not "none
view_df = df.query('tag != "none"')

# Pivot 
view_df = (view_df.pivot(index='name_room', columns='tag',
                        values='pointvalue')
           .reset_index().rename_axis(None, axis=1))

view_df

,name_room,temp1 (RM),temp2 (RMT),valve
0,102,73.39,73.39,12.0
1,112,70.92,70.92,3.0
2,121,70.61,70.61,9.0
3,202,69.60,71.01,15.0
4,203,67.65,67.65,9.0
5,206,69.10,69.10,9.0
6,207,76.93,76.93,9.0
7,212,72.18,72.18,3.0
8,213,71.60,71.60,9.0
9,215,69.11,69.11,9.0


In [85]:
vals = api.building_values_in_range(4, '2017-08-16', '2017-08-19')

In [86]:
vals

,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 00:00:00,150.82,float,DEG F,2017-08-18,00:00
1,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 00:15:00,149.42,float,DEG F,2017-08-18,00:15
2,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 00:30:00,150.36,float,DEG F,2017-08-18,00:30
3,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 00:45:00,150.66,float,DEG F,2017-08-18,00:45
4,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 01:00:00,148.75,float,DEG F,2017-08-18,01:00
5,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 01:15:00,151.03,float,DEG F,2017-08-18,01:15
6,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 01:30:00,149.39,float,DEG F,2017-08-18,01:30
7,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 01:45:00,150.30,float,DEG F,2017-08-18,01:45
8,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 02:00:00,150.58,float,DEG F,2017-08-18,02:00
9,2,13,floatDEG F2,511,BI.LAB.HW.TEMP,2017-08-18 02:15:00,148.83,float,DEG F,2017-08-18,02:15


In [23]:
for idx, row in view_df.iterrows():
    print(row.name_room)

102
112
121
202
203
206
207
212
213
215


In [13]:
view_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name_room</th>\n      <th>temp1 (RM)</th>\n      <th>temp2 (RMT)</th>\n      <th>valve</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>102</td>\n      <td>73.39</td>\n      <td>73.39</td>\n      <td>12.0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>112</td>\n      <td>70.92</td>\n      <td>70.92</td>\n      <td>3.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>121</td>\n      <td>70.61</td>\n      <td>70.61</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>202</td>\n      <td>69.60</td>\n      <td>71.01</td>\n      <td>15.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>203</td>\n      <td>67.65</td>\n      <td>67.65</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>206</td>\n      <td>69.10</td>\n      <td>69.10</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th

In [ ]:
def get_rooms_points(buildings):
    """maps building ids to their points and rooms,
    ie {4:{'rooms':{5}}}"""
    result = {}
    for building_id, name in buildings.items():
        building_data = {
            'rooms': map_rooms(api.building_rooms(building_id)),
            'points': map_points(api.building_points(building_id))
        }
        result[building_id] = building_data
    return result

def map_points(points):
    """Take a pandas df returned from building_points
    and turn it into something easier to use on the frontend
    """
    results = {}
    for index, row in points.iterrows():
        results[row['id']] = row['name'] + '- ' + row['description']
    return results


def map_rooms(rooms):
    """Similar to the map_points function but for rooms"""
    results = {}
    for index, row in rooms.iterrows():
        results[row['id']] = row['name'].replace("_", " ")
    return results